# add Area, SD, HD columns to supersite list
- read in pct_supersite.xlsx
- group by supersite
- extract list of Area, SD, HD


In [1]:
import pandas as pd

In [2]:
# read in pct_supersite.xlsx

cols = ['Pct', 'AreaShort', 'SD', 'HD', 'supersite']
pctss = pd.read_excel('data/pct_supersite.xlsx', usecols=cols)

pctss.info()
pctss

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Pct        193 non-null    int64 
 1   AreaShort  193 non-null    object
 2   SD         193 non-null    int64 
 3   HD         193 non-null    int64 
 4   supersite  193 non-null    object
dtypes: int64(3), object(2)
memory usage: 7.7+ KB


,Pct,AreaShort,SD,HD,supersite
0,2,LM-04,15,19,Trail Ridge MS
1,3,SU-01,18,49,Eldorado K8
2,4,BO-01,15,12,Manhattan MS
3,100,SU-01,18,12,Eldorado K8
4,101,SU-01,18,12,Eldorado K8
...,...,...,...,...,...
188,912,MT-02,15,49,Centennial MS
189,913,MT-02,15,49,Jamestown School
190,914,MT-03,15,49,Allenspark Fire Station
191,915,MT-03,15,49,Lyons Middle Senior


In [3]:
## try create arealist using Pandas functions for Altona MS

# type(pctss[pctss['supersite'] == 'Altona MS']['AreaShort']) # .unique().tolist()

pctss[pctss['supersite'] == 'Altona MS']['AreaShort'].unique().tolist()
# pctss[pctss['supersite'] == 'Altona MS']['SD'].unique().tolist()


['LM-02', 'LM-03']

In [4]:
def collist(s):
    return s.unique().tolist()

ss_areas = pctss.groupby('supersite').agg(
    areas=('AreaShort', collist),
    SDs=('SD', collist),
    HDs=('HD', collist),
    PCTs=('Pct', collist)
)

ss_areas.to_excel('data/supersite_areas.xlsx', index=True)

ss_areas.info()
ss_areas



<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, Allenspark Fire Station  to Ward Community Center
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   areas   20 non-null     object
 1   SDs     20 non-null     object
 2   HDs     20 non-null     object
 3   PCTs    20 non-null     object
dtypes: object(4)
memory usage: 800.0+ bytes


,areas,SDs,HDs,PCTs
supersite,,,,
Allenspark Fire Station,[MT-03],[15],[49],[914]
Altona MS,"[LM-02, LM-03]",[17],[11],"[605, 606, 607, 608, 609, 610, 611, 614, 615, ..."
Burlington Elementary,[LM-01],[17],[11],"[600, 601, 602, 603, 604, 612, 613]"
Casey MS,"[BO-08, BO-07, MT-02]","[18, 15]","[10, 49]","[810, 817, 818, 820, 821, 822, 823, 824, 825, ..."
Centaurus HS,"[LF-01, LF-02]","[17, 18]",[12],"[300, 301, 302, 303, 304, 305, 306, 307, 308, ..."
Centennial MS,"[BO-10, BO-09, MT-02]","[18, 15]","[10, 49]","[803, 804, 805, 806, 807, 811, 812, 813, 814, ..."
Eldorado K8,[SU-01],[18],"[49, 12]","[3, 100, 101, 102, 103, 104, 105, 106]"
Erie MS,[ER-01],[17],[19],"[400, 401, 402, 403, 404, 405, 406, 407, 408]"
Gold Hill School,[MT-02],[15],[49],[909]
